In [1]:
# Difference to v02: all exhange outputs

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from datetime import date

datafile = "./data/2025-06-23_training_data_49755_samples.csv"

In [65]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
# Data loader

def load_and_preprocess_data(filename):
    """Load and preprocess the training data"""
    input_cols = [
        'EX_glc__D_e', 'EX_fru_e', 'EX_lac__D_e', 'EX_pyr_e', 'EX_ac_e',
        'EX_akg_e', 'EX_succ_e', 'EX_fum_e', 'EX_mal__L_e', 'EX_etoh_e',
        'EX_acald_e', 'EX_for_e', 'EX_gln__L_e', 'EX_glu__L_e',
        'EX_co2_e', 'EX_h_e', 'EX_h2o_e', 'EX_nh4_e', 'EX_o2_e', 'EX_pi_e'
    ]

    output_cols = [
        'EX_co2_e_flux', 'EX_h_e_flux', 'EX_h2o_e_flux',
        'EX_nh4_e_flux', 'EX_o2_e_flux', 'EX_pi_e_flux',
        'Biomass_Ecoli_core_flux'
    ]
    df = pd.read_csv(filename)

    # Fill missing inputs with 0 (i.e., not uptaken)
    df[input_cols] = df[input_cols].fillna(0)
    print(f"\nLoaded data with {len(df)} samples from {filename}")

    X = df[input_cols].values.astype(np.float32)
    y = df[output_cols].values.astype(np.float32)

    return X, y, input_cols

In [4]:
X, y, input_cols = load_and_preprocess_data(datafile)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


Loaded data with 49755 samples from ./data/2025-06-23_training_data_49755_samples.csv


In [63]:
X.shape, y.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape

((49755, 20), (49755, 7), (39804, 20), (39804, 7), (9951, 20), (9951, 7))

In [69]:
%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

def f(m, b):
    plt.figure(2)
    x = np.linspace(-10, 10, num=1000)
    plt.plot(x, m * x + b)
    plt.ylim(-5, 5)
    plt.show()

interactive_plot = interactive(f, m=(-2.0, 2.0), b=(-3, 3, 0.5))
output = interactive_plot.children[-1]
# output.layout.height = '200px'
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='m', max=2.0, min=-2.0), FloatSlider(value=0.0, descr…

In [94]:
def plot_data(n):
    plt.figure(figsize=(5, 5))
    plt.stem(np.array(range(1,21)), X[n, :])
    plt.xticks(np.arange(1, 21))
    plt.title(f"Simulation Number {n}")
    plt.grid()
    plt.show()

In [95]:
interactive_plot = interactive(plot_data, n=(0, X.shape[0] - 1))
output = interactive_plot.children[-1]
interactive_plot

interactive(children=(IntSlider(value=24877, description='n', max=49754), Output()), _dom_classes=('widget-int…